In [1]:
# Print every output from a specific cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# Adjusting the screen size
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### Manipulación de metadata de Jobs de BigQuery
 Requiere json file de credenciales

- documentation : https://cloud.google.com/bigquery/docs/managing-jobs#python_2 
- Python Client for Google BigQuery : https://googleapis.dev/python/bigquery/latest/reference.html#job

In [3]:
#! pip freeze
#! pip install --upgrade google-cloud-bigquery

In [4]:
from google.cloud import bigquery
import json
from datetime import datetime, timezone
import pytz

import pandas as pd

timezone = pytz.timezone("UTC")

In [5]:
GCP_CREDENTIALS = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
GCP_CREDENTIALS
BQ_CLIENT = bigquery.Client.from_service_account_json(GCP_CREDENTIALS)

# Jobs todos los proyectos

In [ ]:
# Id de job, generado por BQ cuando ejecuta una query
job_id = 'bquxjob_25d944f8_17961c48028'
#job_proyect_id = 'peya-data-qlty-pro'

In [6]:
list_project = ['peya-bi-tools-pro', 'peya-cx-analytics','peya-delivery-and-support','peya-engineering','peya-food-and-groceries','peya-fraud-and-fintech','peya-growth-and-onboarding','peya-data-origins-pro']


In [ ]:
# datetime object containing current date and time
now = datetime.now()
#fmt = '%Y-%m-%d %H:%M:%S'
#datetime.datetime.strptime(now, fmt)

# Use all_users to include jobs run by all users in the project.
print("Last jobs run by all users:")
for project in list_project:
    BQ_CLIENT.project = project
    for job in BQ_CLIENT.list_jobs(max_results=5, all_users=True, project=project):
        
        if job.state == 'DONE': 
            type(job.created)
            now = datetime.now(timezone.utc)
            print((now - job.created).minutes)

            print('proyect -> {}'.format(project))
            print("Job ID: {id}\n run by user: {user} | State: {state} | {time}\n Query: {query}\n Current duration -> {duration}\n".format(id=job.job_id, user=job.user_email, 
                                                                                                            state=job.state, 
                                                                                                               time=job.created,
                                                                                                                       query=None, duration=None))

In [ ]:
# Jobs by user
print("Last 10 jobs run by all users:")
for job in BQ_CLIENT.list_jobs(max_results=10000, all_users=True, project='peya-food-and-groceries'):
    if job.user_email == 'federico.ramos@pedidosya.com':
        print("\n {id} run by user: {user} | State: {state} | Bytes processed: {bytes} | Labels: {lab}, {time}\n query={query}\n extra: {e}".format(id=job.job_id, 
                                                                                                        user=job.user_email, 
                                                                                                        state=job.state, 
                                                                                                        bytes=job.estimated_bytes_processed, 
                                                                                                        lab=job.labels,
                                                                                                           time=job.created,
                                                                                                                   query=job.query,e=None))

In [ ]:
def Average(lst):
    if len(lst) > 0:
        return sum(lst) / len(lst)
    else:
        return 0

In [ ]:
# Jobs by user

list_project = ['peya-bi-tools-pro', 'peya-cx-analytics','peya-delivery-and-support','peya-engineering','peya-food-and-groceries','peya-fraud-and-fintech','peya-growth-and-onboarding','peya-data-origins-pro']
status = {}
max_results = 1000

for project_id in list_project:
    running=[]
    time_running = []
    pending=[]
    time_pending = []
    current_time = datetime.now(tz=pytz.utc)
    for job in BQ_CLIENT.list_jobs(max_results=max_results, all_users=True, project=project_id, state_filter='pending'):
        difference = current_time - job.created
        execution_time = difference.total_seconds()
        pending.append(job.job_id)
        time_pending.append(execution_time)
    for job in BQ_CLIENT.list_jobs(max_results=max_results, all_users=True, project=project_id, state_filter='running'):
        difference = current_time - job.created
        execution_time = difference.total_seconds()
        running.append(job.job_id)
        time_running.append(execution_time)
            
    status[project_id] = {"Running":len(running), "TimeRunning": Average(time_running), "Pending":len(pending), "TimePending":Average(time_pending)}
    
pd.DataFrame.from_dict(status,orient='index')

In [ ]:
pd.DataFrame.from_dict(status,orient='index')

### Explorar un job especifico

In [ ]:
# https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryPlanEntry.html

In [ ]:
job_id = 'bquxjob_25d944f8_17961c48028'
job_proyect_id = 'peya-data-qlty-pro'

In [ ]:
job = BQ_CLIENT.get_job(job_id=job_id, project=job_proyect_id)

In [ ]:

delta = job.ended - job.started
avg_slots_usage = job.slot_millis / int(delta.total_seconds() * 1000) # milliseconds

print(avg_slots_usage)

In [ ]:
for QueryPlanEntry in job.query_plan:
    print('Entry')
    print(f"""{QueryPlanEntry.name}
        Input Stages:{QueryPlanEntry.} 
        Steps:"""
    )
    for QueryPlanEntryStep  in QueryPlanEntry.steps:
        print(f"""
            {QueryPlanEntryStep.kind}
                {QueryPlanEntryStep.substeps}
        """)


In [ ]:
job.estimated_bytes_processed

In [ ]:
job.total_bytes_processed

In [ ]:
job.query

In [ ]:
# Print selected job properties
print("Details for job {}:".format(job_id))
print(
    "\tType: {}\n\tState: {}\n\tCreated: {}".format(
        job.job_type, job.state, job.created
    )
)

#### Enumera trabajos
https://cloud.google.com/bigquery/docs/managing-jobs#listing_jobs

In [ ]:
# List the 10 most recent jobs in reverse chronological order.
# Omit the max_results parameter to list jobs from the past 6 months.
print("Last 10 jobs:")
for job in BQ_CLIENT.list_jobs(max_results=10,state_filter='running'):  # API request(s)
    print("{} run by user: {} query: {}".format(job.job_id, job.user_email, job.query))

In [ ]:
# The following are examples of additional optional parameters:

# Use min_creation_time and/or max_creation_time to specify a time window.
print("Jobs from the last ten minutes:")
ten_mins_ago = datetime.datetime.utcnow() - datetime.timedelta(minutes=30)
for job in BQ_CLIENT.list_jobs(min_creation_time=ten_mins_ago):
    print("{}".format(job.job_id))

In [13]:
# Use all_users to include jobs run by all users in the project.
print("Last 10 jobs run by all users:")
for job in BQ_CLIENT.list_jobs(max_results=10000, all_users=True, project='peya-food-and-groceries', state_filter='running'):
    if job.user_email == 'federico.ramos@pedidosya.com':
    #if job.state == 'DONE':
        print("\n {id} run by user: {user} | State: {state} | Bytes processed: {bytes} | Labels: {lab}, {time}\n query={query}\n extra: {e}".format(id=job.job_id, 
                                                                                                        user=job.user_email, 
                                                                                                        state=job.state, 
                                                                                                        bytes=job.estimated_bytes_processed, 
                                                                                                        lab=job.labels,
                                                                                                           time=job.created,
                                                                                                                   query=job.query,e=None))

Last 10 jobs run by all users:


In [ ]:
# Use state_filter to filter by job state.
print("Last 10 jobs done:")
for job in BQ_CLIENT.list_jobs(max_results=30, state_filter="DONE"):
    print("{id} run by user: {user} | State: {state} | Bytes processed: {bytes} | Labels: {lab}".format(id=job.job_id, 
                                                                                                        user=job.user_email, 
                                                                                                        state=job.state, 
                                                                                                        bytes=job.estimated_bytes_processed, 
                                                                                                        lab=job.labels))

In [ ]:
# Cancel a job
job = BQ_CLIENT.cancel_job(job_id)